#CS224N Final Project on Question-Answering using SQUAD2.0 Dataset (using Pytorch 1.6.0)

See OneNote Notebook for details

##1. Basic Things

In [1]:
'''
Basic Info:
1. UNKNOWN TOKEN IS 1 (i.e. word not in dictionary/vocabulory)
2. PAD TOKEN IS 0 (padded with 0 for sentences shorter than the max limit)
3. The input data from *.npz file has all the examples shorter than max limit padded with 0.
    The limits are as follows:
    paragraph max limit is 400
    question maximum limit is 50
    answer maximum limit is xx (see in the args section)
4. Colab tips: https://amitness.com/2020/06/google-colaboratory-tips/
'''

!nvidia-smi
!lscpu
!free

Sun Nov 15 22:09:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###To Get More RAM (not needed anymore)

In [ ]:
def get_more_RAM():
    # a = [1.0]*int(1e10) #this is faster
    a = [1.0]
    counter = 1
    while True:
        a = a*10
        counter += 1
# get_more_RAM()

###Install and Import Necessary Packages

In [2]:
!python --version #3.6.9
# !pip install https://download.pytorch.org/whl/cu100/torch-1.3.1%2Bcu100-cp36-cp36m-linux_x86_64.whl
!pip install ujson

Python 3.6.9
     |████████████████████████████████| 184kB 13.7MB/s 


In [3]:
import torch #version 1.3.1 or 1.6.0 or 1.7.0
import numpy as np
import random
print(torch.__version__) #use Torch version 1.3.1

1.7.0+cu101


###PLAYGROUND

In [ ]:
torch.manual_seed(0)
x = torch.rand(2,2)
device = torch.device('cuda', 0)
# torch.cuda.set_device(1)
# device = torch.cuda.device(0)
# x = x.to('cuda') #SEE IF CAN USE TPU
x = x.to(device=device)
print(x, x.type())

# x = x.type(torch.float16) or
x1 = x.to(dtype=torch.float16)
print(x1, x1.type())
print(x1 is x) #false for tensors (but not for nn.Module)

tensor([[0.4963, 0.7682],
        [0.0885, 0.1320]], device='cuda:0') torch.cuda.FloatTensor
tensor([[0.4963, 0.7681],
        [0.0885, 0.1321]], device='cuda:0', dtype=torch.float16) torch.cuda.HalfTensor
False


In [ ]:
#DELETE THIS
from functools import reduce
import time

device = torch.device('cuda', 0)
torch.manual_seed(0)
class DataSet(torch.utils.data.Dataset):
    """docstring for DataSet"""
    def __init__(self, data):
        super(DataSet, self).__init__()
        self.data = data

    def __getitem__(self, index):
        return self.data[0][index], self.data[1][index]
    def __len__(self):
        return len(self.data[0])

in_dim, hid_dim, out_dim = 20, 40, 1
data_size, batch_size, num_epochs = 50, 4, 30
X = torch.rand(data_size, in_dim)
Y = torch.rand(data_size, out_dim)
# Y[:,0] = (X*X).sum(-1)
# Y[:,0] = (torch.sin(X)).sum(-1)
# Y[:,1] = X.sum(-1)**3

dataset = DataSet([X,Y])
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)


class BasicNN(torch.nn.Module):
    """docstring for ClassName"""
    def __init__(self, in_dim, hid_dim, out_dim):
        super(BasicNN, self).__init__()
        self.in_dim = in_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        self.m1 = torch.nn.Linear(in_dim, hid_dim)
        self.lyr_norm1 = torch.nn.LayerNorm([hid_dim], False)
        self.m2 = torch.nn.Linear(hid_dim, hid_dim)
        self.lyr_norm2 = torch.nn.LayerNorm([hid_dim], False)
        self.m3 = torch.nn.Linear(hid_dim, out_dim)
        self.conv = torch.nn.Conv1d(10,10,3,stride=1, groups=2)
        self.reset()

    def forward(self, x):
        y = self.m1(x)
        y = torch.nn.functional.relu(y)
        y = self.lyr_norm1(y)
        y = self.m2(y)
        y = torch.nn.functional.relu(y)
        y = self.lyr_norm2(y)
        y = self.m3(y)
        return y


    def reset(self):
        '''
        Initialize with Glorot initialization
        '''
        # z = list(self.children()) #not recursive
        # print(list(z[0].modules()), print(type(self.lyr_norm1)))
        with torch.no_grad():
            for m in self.modules(): #self.modules() will recursively return all the modules in the network
                if type(m) in [torch.nn.modules.linear.Linear, torch.nn.modules.normalization.LayerNorm]:
                    dims = reduce(lambda x,y: x+y, m.weight.shape)
                    rng = torch.sqrt(torch.tensor(6.0/(dims)))
                    m.weight.uniform_(-rng, rng)
                    # print(dims, type(m), m.weight.shape)
                elif type(m) == torch.nn.modules.conv.Conv1d:
                    dims = reduce(lambda x,y: x*y, m.weight.shape[1:]) + m.weight.shape[0]
                    rng = torch.sqrt(torch.tensor(6.0/(dims)))
                    m.weight.uniform_(-rng, rng)
                    # print(dims, type(m), m.weight.shape)
                # print(m, type(m))


def Training_Func(data_loader):
    model = BasicNN(in_dim, hid_dim, out_dim)
    model = model.to(device)
    
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # optimizer = torch.optim.Adadelta(model.parameters(), lr=0.2)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda e: 1.0/min(max(2, e/15*2)-1, 2)) #start reducing after 10 epochs then reduce by a maximum of 3x

    model.train()
    with torch.enable_grad():
        strt = time.time()
        losses = []
        scaler = torch.cuda.amp.GradScaler()
        for e in range(num_epochs):
            tot_loss = 0
            for x,y in data_loader:
                x = x.to(device)
                y = y.to(device)
                optimizer.zero_grad()

                '''
                y_pred = model(x)
                loss = ((y_pred-y)**2).sum(-1).mean()
                loss.backward()
                optimizer.step()
                '''
                
                with torch.cuda.amp.autocast():
                    y_pred = model(x)
                    loss = ((y_pred-y)**2).sum(-1).mean()

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()


                tot_loss += loss.item()
            losses.append(tot_loss)
            #print(e, tot_loss)
            #print(f"lr is {optimizer.state_dict()['param_groups'][0]['lr']}")
            scheduler.step()
            # break
        end = time.time()
        '''
        plt.plot(losses)
        plt.yscale('log')
        plt.text(10,3, str(end-strt))
        plt.grid(True)
        plt.show()
        '''

    with torch.no_grad():
        print(((model(X.to(device))-Y.to(device))**2).sum())
        #print(model(X), Y)
        print(model)
    return model

strt = time.time()
Training_Func(data_loader)
print(f'Time taken: {time.time()-strt}')
None

tensor(0.8883, device='cuda:0')
BasicNN(
  (m1): Linear(in_features=20, out_features=40, bias=True)
  (lyr_norm1): LayerNorm((40,), eps=False, elementwise_affine=True)
  (m2): Linear(in_features=40, out_features=40, bias=True)
  (lyr_norm2): LayerNorm((40,), eps=False, elementwise_affine=True)
  (m3): Linear(in_features=40, out_features=1, bias=True)
  (conv): Conv1d(10, 10, kernel_size=(3,), stride=(1,), groups=2)
)
Time taken: 1.2718801498413086


###Mount Google Drive

In [4]:
#Create/readin the dataset file
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#To ensure that the Colab Python interpreter can load Python files from within
import sys
import os
FOLDER_NAME = '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final'
sys.path.append(os.path.join(FOLDER_NAME, 'src'))
print(sys.path)

# for auto-reloading external modules
# version 1 (automatically reloads all the time):
# # see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# %load_ext autoreload
# %autoreload 2

# version 2 (reloads when you tell it to i.e. whenever you execute import <module>):
import importlib



os.chdir(FOLDER_NAME)
!pwd
!ls
print()
print('NOTE: ALL THE PATHS ARE REFERENCED FROM THE DIRECTORY WHERE THE COLAB NOTEBOOK IS SAVED BCSE OS.CHDIR() \
IS MADE TO POINT THERE AND ALL .py FILES INSIDE SRC DIRECTORY ARE ONLY CALLED FROM THE COLAB NOTEBOOK AND ALSO BCSE \
ALL THE PATH TO ALL THE SRC/*.PY FILES IS ADDED TO SYS.PATH.')


Mounted at /content/gdrive
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final/src']
/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final
data		 LICENSE    Report.gslides		  src
environment.yml  Readings   save
handouts	 README.md  Squad2_Project_Pytorch.ipynb

NOTE: ALL THE PATHS ARE REFERENCED FROM THE DIRECTORY WHERE THE COLAB NOTEBOOK IS SAVED BCSE OS.CHDIR() IS MADE TO POINT THERE AND ALL .py FILES INSIDE SRC DIRECTORY ARE ONLY CALLED FROM THE COLAB NOTEBOOK AND ALSO BCSE ALL THE PATH TO ALL THE SRC/*.PY FILES IS ADDED TO SYS.PATH.


###Git Version Control

Don't have to do version control on this colab file as well as the report presentation as they have version control availablity by default thru Google Drive.

In [ ]:
!git config --global user.name “[Amit Patel]”
!git config --global user.email “[amitpatel.gt@gmail.com]”
!git config --global color.ui auto
!git config -l

initialize_code_to_git = False #only set to true the first time
if initialize_code_to_git:
    !git init
    !git status
    !git add --all #./src/*.py and .gitignore files only (per .gitignore)
    !git commit -m 'Initial Commit on 07/20/20'

user.name=“[Amit
user.email=“[amitpatel.gt@gmail.com]”
color.ui=auto
core.repositoryformatversion=0
core.filemode=true
core.bare=false
core.logallrefupdates=true
core.symlinks=false


In [ ]:
perform_new_commit = True #only set to true when want to commit new changes
if perform_new_commit:
    !git status
    !git add --all #and .gitignore files only (per .gitignore)
    !git commit -m '11/04/20: Added data shuffling an other things' #PUT A DESCRIPTIVE MESSAGE

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/args.py
	modified:   src/train.py
	modified:   src/train_utils.py
	modified:   src/util.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Delete_Squad2_Project_Pytorch.ipynb
	a1b2x5.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[master b223d21] 11/04/20: Added data shuffling an other things
 6 files changed, 49 insertions(+), 15 deletions(-)
 create mode 100644 Delete_Squad2_Project_Pytorch.ipynb
 create mode 100644 a1b2x5.ipynb


In [ ]:
#change the commmit message
# !git add src/BiDAF.py
# !git commit --amend -m '08/23/20: Added reset() in Bidaf.py and QANet.py'

In [ ]:
# #get config info
# !git config --list

In [ ]:
#Note: don't add anything other than source files
!git status
print('='*100,'\n')
!git log #shows commits for current branch only
print('='*100,'\n')
!git log --stat -M #shows commits for current branch only

In [ ]:
# !git checkout master

In [ ]:
# # view diff between different commits
# !git reset HEAD * #to undo any local changes
# !git diff --cache #diff on staged (i.e. git added) changes
# !git diff #diff on unstaged changes
# !git difftool --tool-help #can use these tools with diff
# !git diff a5b031cb:src/QANet.py f94e67c3:src/QANet.py
# !git difftool --tool=vimdiff 574ce300:src/QANet.py f94e67c3:src/QANet.py
# !git diff a5b031cb:src/BiDAF.py f94e67c3:src/models.py
# !git diff a5b031cb:src/QANet.py src/QANet.py
# !git show
# !git diff ./src/QANet_xfmr_based.py

In [ ]:
# !git branch #lists all the branches
# !git checkout 455ddd41

# #Create a new branch from an older commit above
# !git status
# !git branch previous_version_of_QANet
# !git checkout previous_version_of_QANet
# !git add src/QANet.py
# !git commit -m "Went back to previous version of QANet"

* master
Note: checking out '455ddd41'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 455ddd4 08/08/20 - 2: Updated Author Info


In [ ]:
# !git branch "without_cmd_line_args"
# !git checkout "without_cmd_line_args"

Switched to branch 'without_cmd_line_args'


In [ ]:
# #https://stackoverflow.com/questions/11628074/how-to-make-git-ignore-my-changes
# !git checkout . #to ignore any uncommited/unstaged changes
# #If you want to keep those changes for future use, you have 2 options:
# !git stash #this will save those changes in a stack. You can apply the changes to your working copy later using git stash pop
# !git diff > changes.patch: This will save those changes to the file changes.patch. If you want to apply them to your working copy you can do so: git apply changes.patch

In [ ]:
# !git branch playground
# !git checkout playground

!git checkout master
# !git checkout previous_version_of_QANet

Switched to branch 'master'


In [ ]:
!git status
!git branch --list

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/QANet_xfmr_based.py

no changes added to commit (use "git add" and/or "git commit -a")
* master
  playground
  previous_version_of_QANet
  without_cmd_line_args


In [ ]:
##How to make git forget about a file that is tracked
##First add the file to .gitignore
##https://stackoverflow.com/questions/1274057/how-to-make-git-forget-about-a-file-that-was-tracked-but-is-now-in-gitignore#:~:text=27%20Answers&text=answer%20was%20accepted%E2%80%A6-,.,that%20are%20already%20being%20tracked.&text=The%20removal%20of%20the%20file,happen%20on%20the%20next%20commit.

# !git rm -r --cached . 
# !git add .
# !git commit -am "Remove ignored files i.e. *.ipynb"

rm '.gitignore'
rm 'Delete_Squad2_Project_Pytorch.ipynb'
rm 'LICENSE'
rm 'README.md'
rm 'a1b2x5.ipynb'
rm 'environment.yml'
rm 'src/BiDAF.py'
rm 'src/QANet.py'
rm 'src/QANet_xfmr_based.py'
rm 'src/args.py'
rm 'src/inference.py'
rm 'src/layers.py'
rm 'src/setup.py'
rm 'src/test.py'
rm 'src/train.py'
rm 'src/train_amp.py'
rm 'src/train_utils.py'
rm 'src/util.py'
[master 8b46047] Remove ignored files i.e. *.ipynb
 4 files changed, 31 insertions(+), 28 deletions(-)
 delete mode 100644 Delete_Squad2_Project_Pytorch.ipynb
 delete mode 100644 a1b2x5.ipynb


In [ ]:
# #Shows a tree like-view of everything in the repo
# !git log --graph --pretty=oneline --abbrev-commit
!git log --oneline --decorate --all --graph

* 8b46047 (HEAD -> master) Remove ignored files i.e. *.ipynb
* b223d21 11/04/20: Added data shuffling an other things
* bc2198a 10/27/20: Some minor modifications
* e0014e3 10/10/20: Added hooks to understand training process
* aa2d046 10/02/20: Added train_utils.py
* ba8a53f 09/15/20: Added train_amp.py and modified some files to that end
* 1032f58 09/11/20: Modified train.py and QANet_xfmr_based.py
* e0ad4b7 08/27/20: Modified to allow parameters to be passed thru the command line interface for train.py and test.py
* c418707 (without_cmd_line_args) 08/27/20: Modified qanet_xfmr_based.py
* 034ea87 08/27/20: Modified .gitignore and Added readme, license, and environment.yml
* 9f80376 modified qanet.py, train.py, and added qanet_xfmr.py
* 5a0f096 08/23/20: Added reset() in Bidaf.py and QANet.py
* 3128800 08/21/20: Modified .gitignore to ignore squad2_project_pytorch*.ipynb
* 599f16c 08/21/20: Minor modifications in train.py
* 6561020 08/18/20: Changed to Adam optimizer and added lr sche

###Move files from gdrive to GCP machine (no need to do this as it's not any faster)

In [ ]:
os.chdir('/content')
os.mkdir('temp_amit_prj')
!ls

gdrive	sample_data  temp_amit_prj


In [ ]:
os.chdir(FOLDER_NAME)
!pwd
!ls

/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final
data  Readings	Report.gslides	save  Squad2_Project_Pytorch.ipynb  src


In [ ]:
# !cp -r src '/content/temp_amit_prj/'
!cp -r data '/content/temp_amit_prj/'

In [ ]:
os.chdir('/content/temp_amit_prj/')
!ls

data  src


In [ ]:
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final/src']

In [ ]:
sys.path.remove(os.path.join(FOLDER_NAME, 'src'))
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
sys.path.append('/content/temp_amit_prj/src')
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/temp_amit_prj/src']

In [ ]:
!pwd

/content/temp_amit_prj


In [ ]:
os.chdir('/content')
!pwd
!rm -r temp_amit_prj/

/content


In [ ]:
os.chdir(FOLDER_NAME)
!pwd

/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/project_squad_final


##2. Arguments/inputs to various functions

In [ ]:
import args
importlib.reload(args)
None
args.get_setup_args()
# args.get_train_args()
# args.get_test_args()

{'ans_limit': 30,
 'answer_file': './data/answer.json',
 'char2idx_file': './data/char2idx.json',
 'char_dim': 64,
 'char_emb_file': './data/char_emb.json',
 'char_limit': 16,
 'dev_eval_file': './data/dev_eval.json',
 'dev_meta_file': './data/dev_meta.json',
 'dev_record_file': './data/dev.npz',
 'dev_url': 'https://github.com/chrischute/squad/data/dev-v2.0.json',
 'glove_dim': 300,
 'glove_num_vecs': 2196017,
 'glove_url': 'http://nlp.stanford.edu/data/glove.840B.300d.zip',
 'include_test_examples': True,
 'para_limit': 400,
 'ques_limit': 50,
 'test_eval_file': './data/test_eval.json',
 'test_meta_file': './data/test_meta.json',
 'test_para_limit': 1000,
 'test_ques_limit': 100,
 'test_record_file': './data/test.npz',
 'test_url': 'https://github.com/chrischute/squad/data/test-v2.0.json',
 'train_eval_file': './data/train_eval.json',
 'train_record_file': './data/train.npz',
 'train_url': 'https://github.com/chrischute/squad/data/train-v2.0.json',
 'word2idx_file': './data/word2idx.

##3. Setup

In [ ]:
import spacy
import setup
importlib.reload(setup)


def setup_main():
    # Get command-line args
    args_ = args.get_setup_args()

    # Download resources
    setup.download(args_)

    # Import spacy language model
    nlp = spacy.blank("en")

    # Preprocess dataset
    args_.train_file = setup.url_to_data_path(args_.train_url)
    args_.dev_file = setup.url_to_data_path(args_.dev_url)
    if args_.include_test_examples:
       args_.test_file = setup.url_to_data_path(args_.test_url)
    glove_dir = setup.url_to_data_path(args_.glove_url.replace('.zip', ''))
    glove_ext = f'.txt' if glove_dir.endswith('d') else f'.{args_.glove_dim}d.txt'
    args_.glove_file = os.path.join(glove_dir, os.path.basename(glove_dir) + glove_ext)
    setup.pre_process(args_, nlp)

# setup_main()

In [ ]:
os.chdir('{}/data'.format(FOLDER_NAME))
!ls -l --block-size=M
os.chdir(FOLDER_NAME)
!pwd

total 7047M
-rw------- 1 root root    1M May 11 23:51 char2idx.json
-rw------- 1 root root    2M May 11 23:51 char_emb.json
-rw------- 1 root root   16M May 11 23:51 dev_eval.json
-rw------- 1 root root    1M May 11 23:51 dev_meta.json
-rw------- 1 root root  174M May 11 23:51 dev.npz
-rw------- 1 root root    3M May 11 23:42 dev-v2.0.json
drwx------ 2 root root    1M May 11 23:32 glove.840B.300d
-rw------- 1 root root 2076M Jan  7  2020 glove.840B.300d.zip
-rw------- 1 root root   14M May 11 23:51 test_eval.json
-rw------- 1 root root    1M May 11 23:51 test_meta.json
-rw------- 1 root root  423M May 11 23:51 test.npz
-rw------- 1 root root    2M May 11 23:42 test-v2.0.json
-rw------- 1 root root  287M May 11 23:51 train_eval.json
-rw------- 1 root root 3795M May 11 23:49 train.npz
-rw------- 1 root root   41M May 11 23:27 train-v2.0.json
-rw------- 1 root root    2M May 11 23:51 word2idx.json
-rw------- 1 root root  219M May 11 23:51 word_emb.json
/content/gdrive/My Drive/Colab Noteb

In [ ]:
# os.chdir('/content/sample_data')
# !ls
# print()
# !cat 'README.md'
# print()
# 
# os.chdir('/content')
# !ls
# print()
# os.chdir(FOLDER_NAME)
# !ls

##4. Utility

In [ ]:
import util
importlib.reload(util)

def utils_playground():
    train_data_path = './data/dev.npz'
    train_dataset = util.SQuAD(train_data_path)
    train_dataset.memory_size()
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True, 
                                               num_workers=0, collate_fn=util.collate_fn)
    print()
    a = train_dataset[0:7]
    print(len(a), a[1].shape)
    print()

    for a in train_dataloader:
        for i in range(len(a)):
            print(a[i].shape)
        #print(a[0][0,:])
        #print(a[1][0,:,:])
        break

    del train_dataset
    del train_dataloader

utils_playground()
import gc
gc.collect()

context_idxs: 19.090808MB
context_char_idxs: 305.452928MB
question_idxs: 2.428008MB
question_char_idxs: 38.848128MB
y1s: 0.047608MB
y2s: 0.047608MB
ids: 0.047608MB
valid_idxs: 7.2e-05MB
Total size: 365.962768MB

7 torch.Size([7, 401, 16])

torch.Size([5, 253])
torch.Size([5, 253, 16])
torch.Size([5, 17])
torch.Size([5, 17, 16])
torch.Size([5])
torch.Size([5])
torch.Size([5])


133

##5. Layers

In [ ]:
import layers
importlib.reload(layers)
None

##6. Models

In [ ]:
import QANet
importlib.reload(QANet)
import QANet_xfmr_based
importlib.reload(QANet_xfmr_based)
import util
importlib.reload(util)
None

#debug
word_vectors = util.torch_from_json("./data/word_emb.json")
print(word_vectors.shape)
mod = QANet.QANet(word_vectors, hidden_size=96)

util.get_model_info(mod)

torch.Size([88749, 300])
emb: 561048 parameters
emb_enc: 87648 parameters
att: 289 parameters
mod_enc_input_prj: 36960 parameters
mod_enc: 467040 parameters
out: 384 parameters


In [ ]:
# !pip install line_profiler
# !kernprof -l -v ./src/QANet_xfmr_based.py
!python ./src/QANet.py

##7. Training

In [ ]:
%matplotlib inline
from IPython.display import display, Image
import glob
for f in glob.glob('./save/*.png'):
    print(f)
    display(Image(f))

In [9]:
# import train
# importlib.reload(train)

'''
CHANGES MADE:
1. REMOVED GRAD CLIPPING IN TRAIN.PY (LINE 134). Did not help
2. Using lr=3e-4, it devF1 starts improving after 20 epochs when the lr starts to decrease to 1e-4 (see zzz-02)
3. Using lr=1e-4 (zzz-03): didn't help much.
4. Added positional encoding inside the encoder (zzz-04) and 5e-4: didn't help much

TODO:
4. Try adding l2_wd and put positional encoding insider the encoder like qanet and not outside like orig_xmfr
5. try different see
'''

# ema_decay = 0.999
name='QANET_xfmr_lrwarmup_posenc_inside_encoder_0p05dropprob_lr1e-3_sinsched' #'train-Baseline'
model_type='QANet_xfmr_based.QANet_xfmr' #QANet.QANet
#load_path = os.path.join('./save/train', 'QANET_xfmr_datashuffle_lrwarmup_posenc_inside_encoder_0p1dropprob-01', 'best.pth.tar')  #None

'''
From QANet Paper:
lr = 1.e-3, l2_wd = 3.e-7, batch = 32, drop_prob = 0.1
'''

!python ./src/train.py --num_epochs=40 --lr=1.0e-3 --l2_wd=0.0 --batch_size=32 \
--drop_prob=0.05 --hidden_size=96 --seed=224 --name=$name --model_type=$model_type


Training the Model...
2020-11-16 03:45:19.608859: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
[11.16.20 03:45:22] Args: {
    "batch_size": 32,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.05,
    "ema_decay": 0.999,
    "eval_steps": 50000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 96,
    "l2_wd": 0.0,
    "load_path": null,
    "lr": 0.001,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "model_type": "QANet_xfmr_based.QANet_xfmr",
    "name": "QANET_xfmr_lrwarmup_posenc_inside_encoder_0p05dropprob_lr1e-3_sinsched",
    "num_epochs": 40,
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/train/QANET_xfmr_lrwarmup_posenc_inside_encoder_0p05dropprob_lr1e-3_sinsched-03",
    "seed": 224,
    "test_

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./save/train/QANET_xfmr_NOdatashuffle_lrwarmup_posenc_inside_encoder_0p05dropprob-01

###Debug QANet

In [ ]:
import util
importlib.reload(util)

import QANet_xfmr_based
importlib.reload(QANet_xfmr_based)

import QANet
importlib.reload(QANet)

import BiDAF
importlib.reload(BiDAF)

##############################
vocab_size = int(1e4)
char_vocab_size = 100
device = util.get_available_devices()[0]
word_vocab = torch.rand(vocab_size, 300).to(device)

# model = QANet_xfmr_based.QANet_xfmr(word_vocab, 96)
model = QANet.QANet(word_vocab, 96)
# model = BiDAF.BiDAF_with_char_embed(word_vocab, 200)
model = model.to(device)
util.get_model_info(model)

# Total size 1211161, Total size 4.844643999999997MB and Tot Num parameters 108 #QANet_xfmr 96dim
# Total size 1152601, Total size 4.610404000000002MB and Tot Num parameters 184 #QANet 96dim

#Total size 13703835, Total size 54.815340000000205MB and Tot Num parameters 376 (QANet)
#Total size 6234605, Total size 24.93842MB and Tot Num parameters 57 (BiDaf with chan embed)

emb.char_embed.weight, torch.Size([1378, 200]), param total dim: 275600, 1.1024MB
emb.char_conv.weight, torch.Size([200, 200, 5]), param total dim: 200000, 0.8MB
emb.char_conv.bias, torch.Size([200]), param total dim: 200, 0.0008MB
emb.proj_combine.weight, torch.Size([96, 500]), param total dim: 48000, 0.192MB
emb.hwy.transforms.0.weight, torch.Size([96, 96]), param total dim: 9216, 0.036864MB
emb.hwy.transforms.0.bias, torch.Size([96]), param total dim: 96, 0.000384MB
emb.hwy.transforms.1.weight, torch.Size([96, 96]), param total dim: 9216, 0.036864MB
emb.hwy.transforms.1.bias, torch.Size([96]), param total dim: 96, 0.000384MB
emb.hwy.gates.0.weight, torch.Size([96, 96]), param total dim: 9216, 0.036864MB
emb.hwy.gates.0.bias, torch.Size([96]), param total dim: 96, 0.000384MB
emb.hwy.gates.1.weight, torch.Size([96, 96]), param total dim: 9216, 0.036864MB
emb.hwy.gates.1.bias, torch.Size([96]), param total dim: 96, 0.000384MB
emb_enc.encoder_blocks_list.0.conv_norm_layers.0.weight, tor

In [ ]:
import time
strt = time.time()
!python ./src/QANet.py
print(time.time()-strt, 'seconds')
QANet.GPU_Memory_Usage()
#12.2 sec and 85% (my version) vs 9.2 sec and 79% (pytorch)

EPOCH: 0
AFTER LOSS.BACKWARD
memory used: 13.1484375 (GiB) i.e. 82.698%
***************
EPOCH: 1
AFTER LOSS.BACKWARD
memory used: 13.154296875 (GiB) i.e. 82.735%
***************
7.043651103973389 seconds
memory used: 0.7626953125 (GiB) i.e. 4.797%


In [ ]:
!python -m torch.utils.bottleneck ./src/QANet_xfmr_based.py

`bottleneck` is a tool that can be used as an initial step for debugging
bottlenecks in your program.

It summarizes runs of your script with the Python profiler and PyTorch's
autograd profiler. Because your script will be profiled, please ensure that it
exits in a finite amount of time.

For more complicated uses of the profilers, please see
https://docs.python.org/3/library/profile.html and
https://pytorch.org/docs/master/autograd.html#profiler for more information.
Running environment analysis...
Running your script with cProfile
Profiling QANet_xfmr's forward call
         4850 function calls (4607 primitive calls) in 0.260 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       87    0.119    0.001    0.119    0.001 {built-in method equal}
      129    0.042    0.000    0.042    0.000 {method 'matmul' of 'torch._C._TensorBase' objects}
        5    0.019    0.004    0.019    0.004 {method 'to' of 'torch._C._TensorBase' 

##Playground

In [ ]:
logits = torch.tensor([[1.0,2.0,3], [-1,2,3]])
mask = torch.tensor([[1,1,0],[1,0,0]])
res = util.masked_softmax(logits, mask, log_softmax=False)
print(res)


res = util.masked_softmax(logits, mask, log_softmax=True)
print(res)

logits = torch.tensor([[1.0,2.0,-1e30], [-1,-1e30,-1e30]])
mask = torch.tensor([[1,1,0],[1,0,0]])

res = torch.nn.functional.softmax(logits,-1)
print(res)
res = torch.nn.functional.log_softmax(logits,-1)
print(res)
print(res*mask)
res.sum(-1,keepdim=True)
res.dim()
torch.ones(2,3)
1e30 * 0

tensor([[0.2689, 0.7311, 0.0000],
        [1.0000, 0.0000, 0.0000]])
tensor([[-1.3133e+00, -3.1326e-01, -1.0000e+30],
        [ 0.0000e+00, -1.0000e+30, -1.0000e+30]])
tensor([[0.2689, 0.7311, 0.0000],
        [1.0000, 0.0000, 0.0000]])
tensor([[-1.3133e+00, -3.1326e-01, -1.0000e+30],
        [ 0.0000e+00, -1.0000e+30, -1.0000e+30]])
tensor([[-1.3133, -0.3133, -0.0000],
        [ 0.0000, -0.0000, -0.0000]])


0.0

In [ ]:
#************soft labelling debug (also check memory consumption)

import time

def soft_labelling_loss(log_p, y, cw_idxs):
    '''
    log_p shape (B,C)
    y shape (B,)
    
    for soft labelling, y_soft = (1-eps)*y + eps*1/C
    Use KLDiv loss as it is similar to the cross entropy loss within a constant
    References: 
    1. https://towardsdatascience.com/label-smoothing-making-model-robust-to-incorrect-labels-2fae037ffbd0
    2. https://arxiv.org/pdf/1906.02629.pdf
    '''

    eps = 0.2
    c_mask = torch.zeros_like(cw_idxs) != cw_idxs #(batch_size, max_c_len)
    c_len = c_mask.sum(dim=-1, keepdim=True) #(batch_size, 1)


    B,C = log_p.shape
    y_soft = torch.ones(B,C).cuda() #(B,C)
    y_soft = y_soft * eps / c_len #y=0 case
    y_soft[torch.arange(B), y] = ((1-eps) + torch.div(eps,c_len)).squeeze() #y=1 case
    y_soft *= c_mask #handle masking
    print('ysoft3', y_soft)

    loss = torch.nn.functional.kl_div(log_p, y_soft, reduction='batchmean') #doesn't work. the loss explodes.
    print(loss)
    #dumy = y_soft.element_size() * y_soft.nelement() / 1e6

    # loss = torch.nn.functional.nll_loss(log_p, y)
    # print(loss)

    loss = (-y_soft * log_p).sum() / B

    return loss


# B, C = 400,400
# logits = torch.rand(B,C).cuda()
# log_smax = torch.nn.functional.log_softmax(logits, dim=1)
# y = torch.randint(0,B,(B,)).cuda()

logits = torch.tensor([[1.0,2.0,-1e30], [-1,-1e30,-1e30]]).cuda()
mask = torch.tensor([[1,1,0],[1,0,0]]).cuda()
# mask = torch.tensor([[1,1,1],[1,1,1]]).cuda() #for debug to disable masking

y = torch.tensor([1,0]).cuda()
log_smax = torch.nn.functional.log_softmax(logits,dim=-1)
print(log_smax)
t1 = time.time()
nll = torch.nn.functional.nll_loss(log_smax, y)
print(time.time()-t1)
t1 = time.time()
myloss = soft_labelling_loss(log_smax, y, mask)
print(time.time()-t1)

print(nll, myloss)



tensor([[-1.3133e+00, -3.1326e-01, -1.0000e+30],
        [ 0.0000e+00, -1.0000e+30, -1.0000e+30]], device='cuda:0')
0.00041604042053222656
ysoft3 tensor([[0.1000, 0.9000, 0.0000],
        [1.0000, 0.0000, 0.0000]], device='cuda:0')
tensor(0.0441, device='cuda:0')
0.0033414363861083984
tensor(0.1566, device='cuda:0') tensor(0.2066, device='cuda:0')


In [ ]:
N = int(1e1)
B, C = 400, 100
tot = torch.tensor(0.0).requires_grad_()
for _ in range(N):
    a = torch.rand(B,C)
    tot = tot + a.sum()
a.element_size() * a.nelement() / 1e6

0.16

In [ ]:
import torch.nn as nn
import torch

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.rnn = nn.RNN(input_size=200, hidden_size=1000, num_layers=1)
        self.linear = nn.Linear(1000, 100)

    def forward(self, x, init):
        x = self.rnn(x, init)[0]
        y = self.linear(x.view(x.size(0)*x.size(1), x.size(2)))
        return y.view(x.size(0), x.size(1), y.size(1))

net = testNet()
init = torch.zeros(1, 4, 1000)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

total_loss = 0.0
for i in range(250): #10000 mini-batch
    input = torch.randn(1000, 4, 200) #Seqlen = 1000, batch_size = 4, feature = 200
    target = torch.zeros((4, 1000), dtype=torch.long)

    optimizer.zero_grad()
    output = net(input, init)
    loss = criterion(output.view(-1, output.size(2)), target.view(-1))
    loss.backward()
    optimizer.step()
    total_loss += loss

print(total_loss)

tensor(48.2064, grad_fn=<AddBackward0>)


##8. Testing (Evaluation)

In [ ]:
# import src.test as test #need to do this as there is a module named test in the std library
# importlib.reload(test)

load_path = './save/train/train-With-Char-Embed_200hiddensize-01/best.pth.tar' #'./save/train/train-Baseline-01/best.pth.tar'
num_visuals = 20 #10 #number of examples to write on tensorboard

!python ./src/test.py --split='dev' --load_path=$load_path --name='dev' \
--num_visuals=$num_visuals --hidden_size=200 --model_type=BiDAF.BiDAF_with_char_embed


[08.28.20 04:54:32] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "hidden_size": 200,
    "load_path": "./save/train/train-With-Char-Embed_200hiddensize-01/best.pth.tar",
    "max_ans_len": 15,
    "model_type": "BiDAF.BiDAF_with_char_embed",
    "name": "dev",
    "num_visuals": 20,
    "num_workers": 4,
    "save_dir": "./save/test/dev-05",
    "split": "dev",
    "sub_file": "submission.csv",
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[08.28.20 04:54:32] Loading embeddings...
[08.28.20 04:54:38] Building model...
[08.28.20 04:54:42] Loading checkpoint from ./save/train/train-With-Char-Embed_200hiddensize-01/best.pth.tar...
[08.28.20 04:54:42] Building datas

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./save/test/dev-01

##9. Inference

In [ ]:
from ujson import load as json_load
import util
# import BiDAF
import QANet_xfmr_based


word_emb_file = "./data/word_emb.json"
# load_path = "./save/train/train-With-Char-Embed-01/best.pth.tar"
load_path = "./save/train/QANET_xfmr_datashuffle_lrwarmup_posenc_inside_encoder_0p05dropprob-06/best.pth.tar"
hidden_size = 96 #100
record_file = './data/dev.npz'
eval_file = './data/dev_eval.json'
use_squad_v2 = True
max_ans_len = 15
have_labels = True
batch_size = 2 #64
device, gpu_ids = util.get_available_devices() #from util.py
batch_size *= max(1, len(gpu_ids))

#get embeddings
word_vectors = util.torch_from_json(word_emb_file) #from util.py

#load model
model = QANet_xfmr_based.QANet_xfmr(word_vectors=word_vectors, hidden_size=hidden_size) #from .py
model = torch.nn.DataParallel(model, gpu_ids)
model = util.load_model(model, load_path, gpu_ids, return_step=False) #from util.py
model = model.to(device)
model.eval()

# Get data loader
dataset = util.SQuAD(record_file, use_squad_v2) #from util.py
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False,num_workers=4, collate_fn=util.collate_fn)  #collate_fn from util.py


pred_dict = {}  # Predictions for viewing/printing
with open(eval_file, 'r') as fh:
    gold_dict = json_load(fh)

In [ ]:
import inference
importlib.reload(inference)

#Dataset
contexts = ['God is God and omnipresent , with no comparison . God is the Best .', 'Life without God is pointless . God is the key .']
questions = ['Who is God ?', 'who is most important ?']
# contexts = ['Alice is a student . Students are people . Alice is from Phoenix . Phoenix is a hot city . Cities are places . If it is snow then it is cold . If a person is from a hot place and it is cold, then she is not happy . Alice is happy .', 
            # 'Alice is a student .']
# questions = ['Is it snow ?', 'Is Bob a student ?']
c_widxs, c_cidxs = inference.get_word_and_char_idxs(contexts)
q_widxs, q_cidxs = inference.get_word_and_char_idxs(questions)
y1s = torch.tensor([[2], [2]])
y2s = torch.tensor([[2], [3]])
ids = torch.tensor([0, 1])
dataset_inference = (c_widxs, c_cidxs, q_widxs, q_cidxs, y1s, y2s, ids)
# inference.SQuAD_inference(dataset_inference)[0]

dataset_inference = inference.SQuAD_inference(dataset_inference)
data_loader_inference = torch.utils.data.DataLoader(dataset_inference, batch_size=1, shuffle=False,num_workers=1, collate_fn=util.collate_fn)

# model_untrained = BiDAF(word_vectors=word_vectors, hidden_size=hidden_size) #from BiDAF.py
# model_untrained = nn.DataParallel(model_untrained, gpu_ids)
# model_untrained = model_untrained.to(device)
# model_untrained.eval()

# model_untrained = BiDAF.BiDAF_with_char_embed(word_vectors=word_vectors, hidden_size=hidden_size) #from BiDAF.py
# model_untrained = torch.nn.DataParallel(model_untrained, gpu_ids)
# model_untrained = model_untrained.to(device)
# model_untrained.eval()

print(inference.idx2word_dict[1])
print(dataset_inference[0][0].grad_fn)

--OOV--
None


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

class AddHooks(object):
    '''
    This is used to analyze the attention matrix
    '''
    def __init__(self, model):
        self.model = model
        self.hooks = {}
        self.sim_mtx = {}

        def func(n):
            def f(m_,i_,o_):
                # print(n, '\n', len(o_), o_[0].shape, o_[1].shape)
                self.sim_mtx[n].append(o_[1])
            return f

        for n,m in model.named_modules():
            if 'mha' in n and 'mha.out_proj' not in n:
                # print(n)
                self.sim_mtx[n] = []
                self.hooks[n] = m.register_forward_hook(func(n))
    
    def remove_hooks(self):
        for k,v in self.hooks.items():
            v.remove()
        self.model = None
        self.hooks = None
        self.sim_mtx = None
    

    def plot_heatmap(self, data, xlabels, ylabels, cmap=None):
        fig, ax = plt.subplots()
        im = ax.imshow(data, cmap=cmap)

        # We want to show all ticks...
        ax.set_xticks(np.arange(len(xlabels)))
        ax.set_yticks(np.arange(len(ylabels)))
        # ... and label them with the respective list entries
        ax.set_xticklabels(xlabels)
        ax.set_yticklabels(ylabels)

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                rotation_mode="anchor")

        # Loop over data dimensions and create text annotations.
        for i in range(len(ylabels)):
            for j in range(len(xlabels)):
                text = ax.text(j, i, round(data[i, j], 1), size=5,
                            ha="center", va="center", color="w")


        # ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
        # ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
        # ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
        # ax.tick_params(which="minor", bottom=False, left=False)

        ax.set_title("Attention Matrix")
        # fig.tight_layout()
        # fig.colorbar(im, ax=ax)
        # plt.show()
        # plt.clf()


    def plot(self, xlabels, ylabels):
        for e,(k,v) in enumerate(self.sim_mtx.items()):
            if k == 'module.emb_enc.encoder_blocks_list.1.self_attention.mha':
                for i in range(len(v)):
                    # print(v[i][0].shape, len(xlabels[i]))
                    data = v[i][0].cpu().numpy()
                    self.plot_heatmap(data, xlabels[i], ylabels[i])
                    name = f'./save/atten_{e}_{i}.png'
                    plt.savefig(name, dpi=500)
                    plt.clf()
        self.remove_hooks()


h = AddHooks(model)

In [ ]:
# inference.inference(model_untrained, dataset_inference, data_loader_inference)
inference.inference(model, dataset_inference, data_loader_inference, device, max_ans_len)
# inference.inference(model, dataset, data_loader)

100%|██████████| 2/2 [00:00<00:00,  6.70it/s]


Printing Inference
**Context:** God is God and omnipresent , with no comparison . God is the Best . 
**Question:** Who is God ?
**GT Answer:** Ground truth answer not available
**Prediction:** God and omnipresent


Printing Inference
**Context:** Life without God is pointless . God is the key . 
**Question:** who is most important ?
**GT Answer:** Ground truth answer not available
**Prediction:** N/A



In [ ]:
labels = zip(contexts, questions)
lbls = []; 
for c,q in labels:
    tmp = [w for w in c.split(' ')]
    tmp = ['--OOV--'] + tmp
    lbls.append(tmp)
    tmp = [w for w in q.split(' ')]
    tmp = ['--OOV--'] + tmp
    lbls.append(tmp)
h.plot(lbls, lbls)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [ ]:
%matplotlib inline
from IPython.display import display, Image
import glob
for f in glob.glob('./save/*.png'):
    print(f)
    display(Image(f))
# h.sim_mtx['module.emb_enc.encoder_blocks_list.0.self_attention.mha'][0].shape

##10. Hugging Face Transformers

1. https://pytorch.org/hub/huggingface_pytorch-transformers/
2. https://github.com/huggingface/pytorch-transformers.git
3. https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/huggingface_pytorch-transformers.ipynb

In [ ]:
!pip install transformers
from transformers import *

     |████████████████████████████████| 665kB 7.4MB/s 
     |████████████████████████████████| 890kB 21.6MB/s 
     |████████████████████████████████| 1.1MB 45.7MB/s 
     |████████████████████████████████| 3.8MB 46.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ca8a846468f3748675bccc9d67da4dccea2c3ade10d03792cbf3c898a4cc097d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#Debug only for 'Tokenize the input'
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

text_1 = "who was the person ?"
text_2 = "the person was me ."

indexed_tokens = tokenizer.encode_plus(text_1, add_special_tokens=False)
print(indexed_tokens)

indexed_tokens = tokenizer.encode_plus(text_1, add_special_tokens=True)
print(indexed_tokens)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


{'input_ids': [1150, 1108, 1103, 1825, 136], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [101, 1150, 1108, 1103, 1825, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# 1. Tokenize the input
import torch
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer" #words that are not in vocabulary seem to be plit into words that are (i.e. henson and puppeteer are split into 2 subwords)

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end and between sentences)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)
print(indexed_tokens, len(indexed_tokens))

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


[101, 2627, 1108, 3104, 1124, 15703, 136, 102, 3104, 1124, 15703, 1108, 170, 16797, 8284, 102] 16


In [ ]:
# 2. USING BERTMODEL TO ENCODE THE INPUT SENTENCE INTO A SEQUENCE OF LAST LAYER HIDDEN-STATES

# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)
encoded_layers.shape #the last dim is the size of hidden dim (or vocab size)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


torch.Size([1, 16, 768])

In [ ]:
# 3. USING MODELWITHLMHEAD TO PREDICT A MASKED TOKEN WITH BERT
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
indexed_tokens[masked_index] = tokenizer.mask_token_id
tokens_tensor = torch.tensor([indexed_tokens])

masked_lm__model = torch.hub.load('huggingface/pytorch-transformers', 'modelWithLMHead', 'bert-base-cased')

with torch.no_grad():
    predictions = masked_lm__model(tokens_tensor, token_type_ids=segments_tensors)

# Get the predicted token
predicted_index = torch.argmax(predictions[0][0], dim=1)[masked_index].item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'Jim'
print(f'A: {torch.argmax(predictions[0], dim=2)}, B: {predictions[0].shape}, C: {len(tokenizer.vocab)}')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


A: tensor([[  119,  2627,  1108,  3104,  1124, 15703,   136,   119,  3104,  1124,
         15703,  1108,   170, 16797,  8284,   119]]), B: torch.Size([1, 16, 28996]), C: 28996


In [ ]:
# 4. USING MODELFORQUESTIONANSWERING TO DO QUESTION ANSWERING WITH BERT

question_answering_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
question_answering_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

# The format is paragraph first and then question
text_1 = "Jim Henson was a puppeteer" #puppeteer is separated into two separate words
text_2 = "Who was Jim Henson ?"
indexed_tokens = question_answering_tokenizer.encode(text_1, text_2, add_special_tokens=True)
print(indexed_tokens, len(indexed_tokens))
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

# Predict the start and end positions logits
with torch.no_grad():
    start_logits, end_logits = question_answering_model(tokens_tensor, token_type_ids=segments_tensors)

print(f'start_logits_shape: {start_logits.shape}, vocab: {len(question_answering_tokenizer.vocab)}')
# get the highest prediction
answer = question_answering_tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
assert answer == "puppeteer"

print(f'Positions {torch.argmax(start_logits)} and {torch.argmax(end_logits)+1}')
# Or get the total loss which is the sum of the CrossEntropy loss for the start and end token positions (set model to train mode before if used for training)
start_positions, end_positions = torch.tensor([12]), torch.tensor([14])
multiple_choice_loss = question_answering_model(tokens_tensor, token_type_ids=segments_tensors, start_positions=start_positions, end_positions=end_positions)

question_answering_tokenizer.decode(13997) #puppet
question_answering_tokenizer.decode(11510) #eer

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


[101, 3958, 27227, 2001, 1037, 13997, 11510, 102, 2040, 2001, 3958, 27227, 1029, 102] 14
start_logits_shape: torch.Size([1, 14]), vocab: 30522
Positions 5 and 7


'# # e e r'

In [ ]:
# 5. USING MODELFORSEQUENCECLASSIFICATION TO DO PARAPHRASE CLASSIFICATION WITH BERT

sequence_classification_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-cased-finetuned-mrpc')
sequence_classification_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased-finetuned-mrpc')

text_1 = "Jim Henson was a puppeteer"
text_2 = "Who was Jim Henson ?"
indexed_tokens = sequence_classification_tokenizer.encode(text_1, text_2, add_special_tokens=True)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

# Predict the sequence classification logits
with torch.no_grad():
    seq_classif_logits = sequence_classification_model(tokens_tensor, token_type_ids=segments_tensors)

print(seq_classif_logits)
predicted_labels = torch.argmax(seq_classif_logits[0]).item()

assert predicted_labels == 0  # In MRPC dataset this means the two sentences are not paraphrasing each other

# Or get the sequence classification loss (set model to train mode before if used for training)
labels = torch.tensor([1])
seq_classif_loss = sequence_classification_model(tokens_tensor, token_type_ids=segments_tensors, labels=labels)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master



(tensor([[ 0.9574, -0.2855]]),)


In [ ]:
!pwd

/content


In [ ]:
# 6. Huggingface-Transformers Quick Tour (without using torch.hub.load(...))

import torch
from transformers import *

# Transformers has a unified API
# for 10 transformer architectures and 30 pretrained weights.
#          Model          | Tokenizer          | Pretrained weights shortcut
MODELS = [(BertModel,       BertTokenizer,       'bert-base-uncased'),
          (OpenAIGPTModel,  OpenAIGPTTokenizer,  'openai-gpt'),
          (GPT2Model,       GPT2Tokenizer,       'gpt2'),
          (CTRLModel,       CTRLTokenizer,       'ctrl'),
          (TransfoXLModel,  TransfoXLTokenizer,  'transfo-xl-wt103'),
          (XLNetModel,      XLNetTokenizer,      'xlnet-base-cased'),
          (XLMModel,        XLMTokenizer,        'xlm-mlm-enfr-1024'),
          (DistilBertModel, DistilBertTokenizer, 'distilbert-base-cased'),
          (RobertaModel,    RobertaTokenizer,    'roberta-base'),
          (XLMRobertaModel, XLMRobertaTokenizer, 'xlm-roberta-base'),
         ]

# To use TensorFlow 2.0 versions of the models, simply prefix the class names with 'TF', e.g. `TFRobertaModel` is the TF 2.0 counterpart of the PyTorch model `RobertaModel`

# Let's encode some text in a sequence of hidden-states using each model:
for model_class, tokenizer_class, pretrained_weights in MODELS:
    # Load pretrained model/tokenizer
    tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
    model = model_class.from_pretrained(pretrained_weights)

    # Encode text
    input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

# Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
                      BertForSequenceClassification, BertForTokenClassification, BertForQuestionAnswering]

# All the classes for an architecture can be initiated from pretrained weights for this architecture
# Note that additional weights added for fine-tuning are only initialized
# and need to be trained on the down-stream task
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
for model_class in BERT_MODEL_CLASSES:
    # Load pretrained model/tokenizer
    model = model_class.from_pretrained(pretrained_weights)

    # Models can return full list of hidden-states & attentions weights at each layer
    model = model_class.from_pretrained(pretrained_weights,
                                        output_hidden_states=True,
                                        output_attentions=True)
    input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
    all_hidden_states, all_attentions = model(input_ids)[-2:]

    # Models are compatible with Torchscript
    model = model_class.from_pretrained(pretrained_weights, torchscript=True)
    traced_model = torch.jit.trace(model, (input_ids,))

    # Simple serialization for models and tokenizers
    model.save_pretrained('./directory/to/save/')  # save
    model = model_class.from_pretrained('./directory/to/save/')  # re-load
    tokenizer.save_pretrained('./directory/to/save/')  # save
    tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load

    # SOTA examples for GLUE, SQUAD, text generation...

##Debugging Data Logger

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

In [ ]:
logger = get_logger('./save/test/dev-01', 'dev')
logger.info(f'Args: xxxxxxxzzzzzzzzzzzzzzzzzzzzzzzz')

In [ ]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/save/test/dev-01')
!ls -a
!cat log.txt
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2')

dev_submission.csv  events.out.tfevents.1579658622.05b922c2c9fc.138.2
cat: log.txt: No such file or directory


##Playground

In [ ]:
# import csv
# !pwd

# sub_path = './save/zzzz.csv'
# with open(sub_path, 'w', newline='', encoding='utf-8') as csv_fh:
#     csv_writer = csv.writer(csv_fh, delimiter=',')
#     csv_writer.writerow(['Id', 'Predicted'])
#     csv_writer.writerow(['Id111', 'Predicted1111'])

In [ ]:
# import tqdm
# import time
# for i in tqdm.tqdm(range(3)):
#     for j in tqdm.tqdm_notebook(range(5)):
#         time.sleep(0.5)

In [ ]:
#create data directory in colab virtual machine
# def copy_json_data_files_from_gdrive_to_colabGCP():
#     os.chdir('/content')
#     !mkdir 'data'

#     !cp '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/data/train-v2.0.json' '/content/data/train-v2.0.json'
#     !cp '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/data/test-v2.0.json' '/content/data/test-v2.0.json'
#     !cp '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/data/dev-v2.0.json' '/content/data/dev-v2.0.json'

#     os.chdir('/content/data')
#     !ls
# copy_json_data_files_from_gdrive_to_colabGCP()


# #check all the data files
# os.chdir('/content/data')
# !ls -l --block-size=M

# import shutil

# def copy_important_data_files_from_colabGCP_to_gdrive():
#     #copy important data files to gdrive
#     list_of_files = ['char2idx.json', 'char_emb.json', 'dev_eval.json', 'dev_meta.json', 'dev.npz', 'test_eval.json', 'test_meta.json', 'test.npz',
#                      'train_eval.json', 'train.npz', 'word2idx.json', 'word_emb.json']
#     for fname in list_of_files:
#         src = '/content/data/' + fname
#         dest = '/content/gdrive/My Drive/Colab Notebooks/Deep_Learning/CS224N_2019/Project_Squad2/data/' + fname
#         shutil.copyfile(src, dest)
#         #!cp src dest

# # copy_important_data_files_from_colabGCP_to_gdrive()

# #shutil.rmtree('/content/data') #remove non-empty directory (to save disk space)
